In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import load_model,save_model,Sequential
from keras.layers import Activation,Conv2D,Dense,Dropout,Flatten,MaxPooling2D
from keras.preprocessing import image
from keras.utils import to_categorical
import random

In [ ]:
#Params
train_dir = '../dataset/train'
test_dir = '../dataset/test'
train_data = []
test_data = []
X_train = []
y_train = []
X_test = []
y_test = []
shape = [24,24,3]
Categories = ['close','open']

In [ ]:
def prepare_train_data():
    for category in Categories:
        path = os.path.join(train_dir,category)
        cat_idx = Categories.index(category)
        for img in os.listdir(path):
            try:
                image = cv2.imread(os.path.join(path,img))
                image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(24,24))
                train_data.append([image,cat_idx])
            except Exception as e:
                pass
prepare_train_data()

In [ ]:
random.shuffle(train_data)

In [ ]:
for features,labels in train_data:
    X_train.append(features)
    y_train.append(labels)

In [ ]:
X_train = np.array(X_train).reshape(-1,shape[0],shape[1],shape[2])
X_train = X_train/255
y_cat_train = to_categorical(y_train,2)

In [ ]:
def prepare_test_data():
    for category in Categories:
        path = os.path.join(test_dir,category)
        cat_idx = Categories.index(category)
        for img in os.listdir(path):
            try:
                image = cv2.imread(os.path.join(path,img))
                image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(24,24))
                test_data.append([image,cat_idx])
            except Exception as e:
                pass
prepare_test_data()

In [ ]:
random.shuffle(test_data)

In [ ]:
for features,labels in test_data:
    X_test.append(features)
    y_test.append(labels)

In [ ]:
X_test = np.array(X_test).reshape(-1,shape[0],shape[1],shape[2])
X_test=X_test/255
y_cat_test = to_categorical(y_test,2)

In [ ]:
def train_model():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(24,24,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(1,1)))
    model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(24,24,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(1,1)))
    model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(24,24,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(1,1)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(X_train,y_cat_train,epochs=15,validation_split=0.2)
    return model
model = train_model()

In [ ]:
def save_model(model):
    model.save('SafeDrive.h5')
save_model(model)